In [29]:
import numpy as np
import uproot
import pandas as pd
import awkward as ak
from tqdm import tqdm

In [30]:
with uproot.open('atmospherics_prod_1M_events_cafs_hadd_with_weights.root') as f:
    weights = f['weights'].arrays(library='pd')
    weights['nuPDG'] = ak.flatten(f['cafTree/rec/mc/mc.nu.pdg'].array())
    weights['Ev'] = ak.flatten(f['cafTree/rec/mc/mc.nu.E'].array())
    weights['NuMomX'] = ak.flatten(f['cafTree/rec/mc/mc.nu.momentum.x'].array())
    weights['NuMomY'] = ak.flatten(f['cafTree/rec/mc/mc.nu.momentum.y'].array())
    weights['NuMomZ'] = ak.flatten(f['cafTree/rec/mc/mc.nu.momentum.z'].array())
    weights['coszen'] = weights['NuMomY'] / np.sqrt(weights['NuMomX']**2 + weights['NuMomY']**2 + weights['NuMomZ']**2)

    weights['nue_unosc_w'] = weights['xsec'] * weights['nue_w']
    weights['numu_unosc_w'] = weights['xsec'] * weights['numu_w']
df = weights

In [31]:
#Get OscProb
import ROOT
ROOT.gSystem.Load('/home/seave/anaconda3/OscProb/lib/libOscProb.so')

1

In [32]:
def ConfigurePMNS(mh):
    
    myPMNS = ROOT.OscProb.PMNS_NSI()
    prem = ROOT.OscProb.PremModel()

    # Set basic oscillation parameters (Best fit as of 2024)
    dm21 = 7.49e-5
    dm31 = 2.513e-3 if mh > 0 else -2.510e-3
    th12 = np.arcsin(np.sqrt(0.308))
    th13 = np.arcsin(np.sqrt(0.02215)) if mh > 0 else np.arcsin(np.sqrt(0.02224));
    th23 = np.arcsin(np.sqrt(0.470)) if mh > 0 else np.arcsin(np.sqrt(0.562));
    dcp = 212*(np.pi/180) if mh> 0 else 285*(np.pi/180)

    myPMNS.SetMix(th12, th23, th13, dcp);
    myPMNS.SetDeltaMsqrs(dm21, dm31); 

    return myPMNS, prem

In [33]:
def Get_Prob(E, coszen, mh, flav_in, flav_out, nsiDict, is_antinu):
  
  myPMNS, prem = ConfigurePMNS(mh)
    
  if is_antinu:
    myPMNS.SetIsNuBar(True)
  #Set epsilon and delta values to given nsi param
  if nsiDict is not None:
      nsi_param = list(nsiDict.keys())[0]
      nsi_value, delta = nsiDict[nsi_param] 
      
      # Reset ONLY the ones you're not using
      if nsi_param == 'eps_emu':
          myPMNS.SetEps_etau(0, 0)
          myPMNS.SetEps_mutau(0, 0)
          myPMNS.SetEps_emu(nsi_value, delta)
      elif nsi_param == 'eps_etau':
          myPMNS.SetEps_emu(0, 0)
          myPMNS.SetEps_mutau(0, 0)
          myPMNS.SetEps_etau(nsi_value, delta)
      elif nsi_param == 'eps_mutau':
          myPMNS.SetEps_emu(0, 0)
          myPMNS.SetEps_etau(0, 0)
          myPMNS.SetEps_mutau(nsi_value, delta)
      else:
          print("Invalid NSI Parameter")

  #Get total length of path and set path through earth
  L = prem.GetTotalL(coszen)
  prem.FillPath(coszen);
  myPMNS.SetPath(prem.GetNuPath());

  prob =  myPMNS.Prob(flav_in, flav_out, E, L)
  return prob

In [34]:
def Get_Expected(df, mh, flav_out, nsiDict):
    total_weighted = 0.0
    
    for _, row in df.iterrows():
        E = row['Ev']
        cz = row['coszen']
        nu_pdg = row['nuPDG']
        is_antinu = (nu_pdg < 0)
        flav_in = abs(nu_pdg)  # Use 12, 14, 16 for ν_e, ν_μ, ν_τ

        # Pick the correct unoscillated weight
        if flav_in == 12:  # ν_e or ν̄_e
            weight = row['nue_unosc_w']
            flav_in_code = 0
        elif flav_in == 14:  # ν_μ or ν̄_μ
            weight = row['numu_unosc_w']
            flav_in_code = 1
        else:
            continue 

        # Compute oscillation probability
        prob = Get_Prob(
            E, cz, mh,
            flav_in_code, flav_out,
            nsiDict,
            is_antinu
        )
        total_weighted += weight * prob

    return total_weighted


In [35]:
def Make_NSI_Dict(eps_name, eps_val, delta):
    nsiDict = {eps_name: [eps_val, delta]}
    return nsiDict

In [28]:
#Get progress bar, will likely take some time
from tqdm import tqdm
import time

eps_vals = np.linspace(0, 0.5, 5)
delta_vals = np.linspace(0, 2*np.pi, 5)
chi2_map = np.zeros((len(eps_vals), len(delta_vals)))

mh = +1  # or -1 for inverted hierarchy
flav_out = 1  # observed flavor (e.g., νμ)

# Compute standard events
N_std = Get_Expected(df, mh, flav_out, nsiDict = None)
print(N_std)
# Scan grid
for i, eps in enumerate(tqdm(eps_vals, desc="Scanning ε")):
    for j, delta in enumerate(delta_vals):
        nsiDict = Make_NSI_Dict("eps_emu", eps, delta)
        N_nsi = Get_Expected(df, mh, flav_out, nsiDict)
        print(N_nsi)
        chi2_map[i, j] = ((N_nsi - N_std)**2) / (N_std)


46790.35503776888


Scanning ε:   0%|                                                                                 | 0/5 [00:00<?, ?it/s]

46790.35503776888
46790.35503776888


Scanning ε:   0%|                                                                                 | 0/5 [00:52<?, ?it/s]


KeyboardInterrupt: 

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

